In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from src.paths import TRANSFORMED_DATA
from src.data_extraction import load_raw_data
from src.data_transformations import clean_raw_data, transform_cleaned_data_into_ts_data, transform_ts_into_training_data

## Get the raw data

In [2]:
trips_2023 = list(
        load_raw_data(
            year=2023, months = list(range(1,12))
        )
)

The file 202301-divvy-tripdata.zip is already in local storage
The file 202302-divvy-tripdata.zip is already in local storage
The file 202303-divvy-tripdata.zip is already in local storage
The file 202304-divvy-tripdata.zip is already in local storage
The file 202305-divvy-tripdata.zip is already in local storage
The file 202306-divvy-tripdata.zip is already in local storage
The file 202307-divvy-tripdata.zip is already in local storage
The file 202308-divvy-tripdata.zip is already in local storage
The file 202309-divvy-tripdata.zip is already in local storage
The file 202310-divvy-tripdata.zip is already in local storage
The file 202311-divvy-tripdata.zip is already in local storage


In [3]:
# Form a dataframe by concatenating all the available months of 2023
trips = pd.concat(trips_2023)

## Clean the data

In [4]:
# Clean the data
trips = clean_raw_data(trips)

In [5]:
trips

,start_time,stop_time,start_latitude,start_longitude,stop_latitude,stop_longitude
0,2023-01-21 20:05:42,2023-01-21 20:16:33,41.924074,-87.646278,41.930000,-87.640000
1,2023-01-10 15:37:36,2023-01-10 15:46:05,41.799568,-87.594747,41.809835,-87.599383
2,2023-01-02 07:51:57,2023-01-02 08:05:11,42.008571,-87.690483,42.039742,-87.699413
3,2023-01-22 10:52:58,2023-01-22 11:01:44,41.799568,-87.594747,41.809835,-87.599383
4,2023-01-12 13:58:01,2023-01-12 14:13:20,41.799568,-87.594747,41.809835,-87.599383
...,...,...,...,...,...,...
362513,2023-11-24 08:39:27,2023-11-24 08:47:03,41.936497,-87.647539,41.935775,-87.663600
362514,2023-11-06 09:07:20,2023-11-06 09:10:00,41.877726,-87.654787,41.877642,-87.649618
362515,2023-11-10 19:35:30,2023-11-10 19:44:28,41.943687,-87.648855,41.935775,-87.663600
362516,2023-11-27 09:11:23,2023-11-27 09:13:23,41.877726,-87.654787,41.877642,-87.649618


## Define datasets

In [6]:
starts = trips[
    ["start_time", "start_latitude", "start_longitude"]
]

stops = trips[
    ["stop_time", "stop_latitude", "stop_longitude"]
]

## Transform datasets into time series

In [7]:
agg_starts, agg_stops = transform_cleaned_data_into_ts_data(start_df = starts, stop_df = stops)

/home/kobina/Desktop/ML/End-to-End Projects/Actualised/cyclistic-bike-sharing-data/src/data_transformations.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f"{scenario}_hour"] = data.loc[:, f"{scenario}_time"].dt.floor("H")


This might take a moment. Doing some preliminary setup


100%|██████████| 5489029/5489029 [00:26<00:00, 206505.53it/s]


Matching up approximate locations with generated IDs


1832it [00:00, 1750333.70it/s]
/home/kobina/Desktop/ML/End-to-End Projects/Actualised/cyclistic-bike-sharing-data/src/data_transformations.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f"{scenario}_hour"] = data.loc[:, f"{scenario}_time"].dt.floor("H")


This might take a moment. Doing some preliminary setup


100%|██████████| 5489029/5489029 [00:25<00:00, 216705.54it/s]


Matching up approximate locations with generated IDs


1240it [00:00, 2006534.32it/s]
100%|██████████| 1199/1199 [00:24<00:00, 49.49it/s]


In [13]:
agg_stops.to_parquet(path = TRANSFORMED_DATA/"agg_stops.parquet")
agg_starts.to_parquet(path =TRANSFORMED_DATA/"agg_starts.parquet")

## Transform time series data into training data

### Start data

In [2]:
agg_starts = pd.read_parquet(path=TRANSFORMED_DATA/"agg_starts.parquet")

In [3]:
start_features, start_target = transform_ts_into_training_data(
    ts_data=agg_starts,
    start_or_stop="start",
    input_seq_len=24*28*1, 
    step_size=24
)

print(f"{start_features.shape=}")
print(f"{start_target.shape=}")

100%|██████████| 1372/1372 [04:47<00:00,  4.78it/s]


start_features.shape=(419832, 674)
start_target.shape=(419832,)


### Stop data